In [24]:
import os
import pandas as pd
from unidecode import unidecode
import numpy as np
import re
import importlib
import global_settings
import hms_mh_cleansing_before_unification_function
import hms_mh_unification_after_cleansing_function


importlib.reload(global_settings)
importlib.reload(hms_mh_cleansing_before_unification_function)
importlib.reload(hms_mh_unification_after_cleansing_function)

<module 'hms_mh_unification_after_cleansing_function' from '/data/workdirectory/03. HMS unification/5. 20250320/MC/hms_mh_unification_after_cleansing_function.py'>

In [25]:
setts = global_settings.Settings()
cleansing_function = hms_mh_cleansing_before_unification_function.cleansing_function
unification_old_new_function = hms_mh_unification_after_cleansing_function.unification_old_new_function

province = pd.read_csv(setts.input_folder + "/province.csv",dtype=str)
district = pd.read_csv(setts.input_folder + "/district.csv",dtype=str)
convert_phonenumber = pd.read_csv(setts.input_folder + "/convert_phonenumber.csv",dtype=str)

Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp1
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp2
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/output


In [26]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [27]:
case = 'case2'
# case = 'match_vin_only'
output_path = setts.output_folder + '/cluster_' + case
create_folder_if_not_exists(output_path)
print(output_path)

df_combi = pd.read_csv(setts.tmp2_folder + '/hms_mh_%s.csv' % case,dtype=str).fillna('')
df_combi

/data/workdirectory/03. HMS unification/5. 20250320/MC/output/cluster_case2


,unification_id,customer_code,vin,customer_name,customer_address,mobile_phone,district,city,null_case,cluster
0,,HMS_1-44IJYBQ,RLHJK1408RZ256717,,NGŨ ĐOÀI ĐIỆP NÔNG ĐÀ NẴNG,0906420280,other,danang,2,0
1,,HMS_1-5PW6JXA,RLHJK0353RY177226,,MỴ HOÀ KIM BÔI HÒA BÌNH HÒA BÌNH,0328189456,hoabinh,hoabinh,2,0
2,,HMS_1-5P6M2AC,RLHJA3930RY175668,,SỐ 3 TRẦN QUỐC TOẢN QUANG TRUNG HƯNG YÊN,0387166866,other,hungyen,2,0
3,,HMS_1-2SZSVW0,RLHJK1244RZ010754,,ĐẶNG XÁ CƯƠNG CHÍNH TIÊN LỮ HƯNG YÊN HƯNG YÊN,0394745947,hungyen,hungyen,2,0
4,,HMS_1-6HZDPQG,RLHJA3932SY012008,,SUNG KÉP IA KLA GIA LAI,0338996903,other,gialai,2,0
...,...,...,...,...,...,...,...,...,...,...
4362,5344aa99de86928db320b4020035fa7b9df4f20b2d0a78...,MH_4015891,RLHJF630XFZ221112,,BÌNH ĐỊNH,0983370203,other,binhdinh,2,0
4363,a27cb0e0b1a67fe8748f8ee3bf5dd4a76c44468095b604...,MH_6203722,RLHJF180XAY137656,,AH NGA SƠN BÌNH ĐỊNH,0935524387,other,binhdinh,2,0
4364,a6c50cd6afd0bcf4838980a6b02c80640f25b0a8bf5a33...,MH_8587693,RLHJF6325JZ024407,,QUY NHƠN HẬU LỘC BÌNH ĐỊNH,0763838177,other,binhdinh,2,0
4365,fa0eb5e035c9091f5d895a2bff715e90e43ab2e16e2dc6...,MH_1788922,RLHJF5131GY022609,,HẬU LỘC BÌNH ĐỊNH,0829365747,other,binhdinh,2,0


In [28]:
cluster_list = df_combi['cluster'].astype(int).drop_duplicates().sort_values().tolist()
print(cluster_list)

[0]


In [29]:
def generate_key(vin, name, address, phone, district, city, cut=False):
    if cut:
        name = hms_mh_cleansing_before_unification_function.clean_name_old_data(name)
        address = hms_mh_cleansing_before_unification_function.clean_address_old_data(address, district, city)
    
    return (vin+name+address+phone + district + city).lower().replace(' ', '')

df_combi['key'] = df_combi.apply(lambda x: generate_key(x['vin'], 
                                                          x['customer_name'], 
                                                          x['customer_address'], 
                                                          x['mobile_phone'], 
                                                          x['district'], 
                                                          x['city'], cut=True), axis=1)

In [30]:
df_old = df_combi[df_combi['unification_id'].str.len() != 0]
df_new = df_combi[df_combi['unification_id'].str.len() == 0]

print(len(df_old))
print(len(df_new))

4353
14


In [31]:
# Go through each cluster
for cluster in cluster_list:
    print('=' * 50)
    print(f'Unifying case "{case}" cluster "{cluster}"')
    
    df_combi_cluster = df_combi[df_combi['cluster'] == str(cluster)]
    output_name = output_path + '/hms_mh_old_new_unified_cluster_%s.csv' % str(cluster)
    
    # Extract new and old data of cluster
    df_old_cluster = df_old[df_old['cluster'] == str(cluster)]
    df_new_cluster = df_new[df_new['cluster'] == str(cluster)]

    
    # Check if there are new data in cluster
    if len(df_new_cluster) == 0:
        print(f'No new data in cluster "{cluster}" of case "{case}", skipping')
        continue
        
    # Clean old & new data
    df_old_cluster, df_new_cluster = cleansing_function(df_old_cluster, 
                                                        df_new_cluster, 
                                                        province, district, 
                                                        convert_phonenumber)
    
    
    print('Number of rows in df_old: {:,}'.format(df_old_cluster.shape[0]))
    print('Number of rows in df_new: {:,}'.format(df_new_cluster.shape[0]))
    print('Expected number of rows after unification: {:,}'.format(df_new_cluster.shape[0] + df_old_cluster.shape[0]))
    
    # Unification
    df_unified_cluster = unification_old_new_function(df_old_cluster, 
                                                      df_new_cluster, 
                                                      convert_phonenumber)
    
    df_unified_cluster['key'] = df_unified_cluster.apply(lambda x: generate_key(x['vin_cleansing'], 
                                                                  x['name_cleansing'], 
                                                                  x['address_cleansing'], 
                                                                  x['phone_cleansing'], 
                                                                  x['district_group'], 
                                                                  x['city_group']), axis=1)
    
    df_unified_cluster = pd.merge(left = df_combi_cluster, right = df_unified_cluster, how='left', on='key')
    df_unified_cluster = df_unified_cluster[['unification_id_y', 'customer_code', 'vin', 'customer_name', 
                                           'customer_address', 'mobile_phone', 
                                           'name_cleansing', 'address_cleansing', 'phone_cleansing', 
                                           'district', 'city', 'cluster']]
    df_unified_cluster = df_unified_cluster.rename(columns = {'unification_id_y' : 'unification_id'})
    df_unified_cluster.to_csv(output_name, index=False)
    print('Actual number of rows after unification: {:,}'.format(df_unified_cluster.shape[0]))
    print('Output saved:', output_name)

Unifying case "case2" cluster "0"
Number of rows in df_old: 4,353
Number of rows in df_new: 14
Expected number of rows after unification: 4,367


Processing rows: 100%|██████████| 14/14 [00:00<00:00, 86.93it/s]


Actual number of rows after unification: 4,367
Output saved: /data/workdirectory/03. HMS unification/5. 20250320/MC/output/cluster_case2/hms_mh_old_new_unified_cluster_0.csv


In [32]:
df_unified_cluster

,unification_id,customer_code,vin,customer_name,customer_address,mobile_phone,name_cleansing,address_cleansing,phone_cleansing,district,city,cluster
0,3b55fbf6c0c7829c577d592980efdfecf97f95c24f11a2...,HMS_1-44IJYBQ,RLHJK1408RZ256717,,NGŨ ĐOÀI ĐIỆP NÔNG ĐÀ NẴNG,0906420280,,ngudoaidiepnong,0906420280,other,danang,0
1,525bfd3778010140ce740b2bc29a924ec99e194bb6e651...,HMS_1-5PW6JXA,RLHJK0353RY177226,,MỴ HOÀ KIM BÔI HÒA BÌNH HÒA BÌNH,0328189456,,myhoakimboi,0328189456,hoabinh,hoabinh,0
2,957064b189cdfc2c0d0ac73abdf276dd33e625c1a92322...,HMS_1-5P6M2AC,RLHJA3930RY175668,,SỐ 3 TRẦN QUỐC TOẢN QUANG TRUNG HƯNG YÊN,0387166866,,3tranquoctoanquangtrung,0387166866,other,hungyen,0
3,4084afd12431778b9d84b6444b9786bcb5a25cf5043a04...,HMS_1-2SZSVW0,RLHJK1244RZ010754,,ĐẶNG XÁ CƯƠNG CHÍNH TIÊN LỮ HƯNG YÊN HƯNG YÊN,0394745947,,dangcuongchinhtienlu,0394745947,hungyen,hungyen,0
4,9923a6b3b0d70b33dae1e09a61621209dd3cdfdded74ba...,HMS_1-6HZDPQG,RLHJA3932SY012008,,SUNG KÉP IA KLA GIA LAI,0338996903,,sungkepiakla,0338996903,other,gialai,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4362,5344aa99de86928db320b4020035fa7b9df4f20b2d0a78...,MH_4015891,RLHJF630XFZ221112,,BÌNH ĐỊNH,0983370203,,,0983370203,other,binhdinh,0
4363,a27cb0e0b1a67fe8748f8ee3bf5dd4a76c44468095b604...,MH_6203722,RLHJF180XAY137656,,AH NGA SƠN BÌNH ĐỊNH,0935524387,,ahngason,0935524387,other,binhdinh,0
4364,a6c50cd6afd0bcf4838980a6b02c80640f25b0a8bf5a33...,MH_8587693,RLHJF6325JZ024407,,QUY NHƠN HẬU LỘC BÌNH ĐỊNH,0763838177,,quynhonhauloc,0763838177,other,binhdinh,0
4365,fa0eb5e035c9091f5d895a2bff715e90e43ab2e16e2dc6...,MH_1788922,RLHJF5131GY022609,,HẬU LỘC BÌNH ĐỊNH,0829365747,,hauloc,0829365747,other,binhdinh,0


In [33]:
# df_unified_cluster_copy = df_unified_cluster.copy()
# df_unified_cluster_copy = df_unified_cluster_copy[['unification_id_y', 'customer_code', 'vin', 'customer_name', 
#                                            'customer_address', 'mobile_phone', 
#                                            'name_cleansing', 'address_cleansing', 'phone_cleansing', 
#                                            'district', 'city', 'cluster']]
# df_unified_cluster_copy = df_unified_cluster_copy.rename(columns = {'unification_id_y' : 'unification_id'})
# df_unified_cluster_copy.to_csv(output_name, index=False)